# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data = pd.read_csv('city_data.csv')
data = city_data.drop(columns=["Unnamed: 0"])
data.head()

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed
0,Barrow,71.29,-156.79,-18.00,77,90,2.10
1,Thompson,55.74,-97.86,-26.00,68,5,3.60
2,Mataura,-46.19,168.86,13.33,98,100,1.34
3,Lompoc,34.64,-120.46,9.49,100,40,3.10
4,Khatanga,71.97,102.50,-38.18,98,2,3.15


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
gmaps.configure(api_key=g_key)

In [12]:
loc_df = data[["Latitude", "Longitude"]]

humidity = data["Humidity"].astype(float)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(loc_df, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [25]:
best_weather = data
best_weather['F_Temperature'] = best_weather.apply(lambda row: (row.Temperature * 9/5)+32, axis = 1) 

#A max temperature lower than 80 degrees but higher than 70.
#Wind speed less than 10 mph.
#Zero cloudiness.
#Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.

temperature = (best_weather["F_Temperature"] <= 80) & (best_weather["F_Temperature"] > 70)
wind_speed = best_weather["Wind Speed"] < 10
cloudiness = best_weather["Cloudiness"] == 0

best_weather = best_weather[temperature & wind_speed & cloudiness]
best_weather

,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Wind Speed,F_Temperature
10,Mossamedes,-15.20,12.15,25.42,83,0,8.43,77.756
16,Cape Town,-33.93,18.42,26.54,54,0,5.70,79.772
18,Moranbah,-22.00,148.05,24.92,80,0,3.28,76.856
42,Bherāmāra,24.02,88.99,23.62,54,0,3.08,74.516
47,Hermanus,-34.42,19.23,24.32,79,0,3.58,75.776
63,Bredasdorp,-34.53,20.04,26.54,48,0,5.70,79.772
97,Port Alfred,-33.59,26.89,22.22,79,0,1.34,71.996
100,Diego de Almagro,-26.37,-70.05,24.52,37,0,3.37,76.136
107,Carnarvon,-24.87,113.63,26.00,78,0,9.30,78.800
120,Ibrā’,22.69,58.53,24.92,20,0,4.67,76.856


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
hotel_df = pd.DataFrame(best_weather, columns=["City", "Latitude", "Longitude"])

hotel_df["Hotel Name"] = ""
hotel_df

,City,Latitude,Longitude,Hotel Name
10,Mossamedes,-15.20,12.15,
16,Cape Town,-33.93,18.42,
18,Moranbah,-22.00,148.05,
42,Bherāmāra,24.02,88.99,
47,Hermanus,-34.42,19.23,
63,Bredasdorp,-34.53,20.04,
97,Port Alfred,-33.59,26.89,
100,Diego de Almagro,-26.37,-70.05,
107,Carnarvon,-24.87,113.63,
120,Ibrā’,22.69,58.53,


In [31]:
#create list for hotels
hotel_name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # make location parameter a variable
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    address = requests.get(base_url, params=params)
    
    # convert to json
    address = address.json()
    try:
        hotel_name.append(address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)
        
hotel_name

['Hotel ibis Styles Iu Namibe',
 'The Crystal',
 'Direct Hotels - Monterey Moranbah',
 nan,
 'The Marine Hermanus',
 'Bredasdorp Country Manor',
 'The Halyards Hotel',
 'hotel luna del desierto',
 'The Gascoyne Hotel',
 'Ibra Hotel',
 'Ormara Beach Resorts/ Bolan Safari',
 'Salalah Gardens Hotel',
 'Hotel Cocomar',
 nan,
 'Sur Plaza Hotel',
 '3G hotel pasni',
 'Kenzo Hotel',
 'Diplomat Hotel',
 'Yadav Dhaba']

In [32]:
hotel_df['Hotel Name']= hotel_name
hotel_df.dropna()
hotel_df.to_csv('hotel_export.csv')

In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [39]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))